In [1]:
!sudo ln -sf /usr/share/zoneinfo/Asia/Kolkata /etc/localtime
!echo "Asia/Kolkata" | sudo tee /etc/timezone

[sudo] password for jovyan: 
[sudo] password for jovyan: 


In [2]:
'''Task is to
Clean columns (trim, fix cases, remove duplicates)

Convert datatypes

Derive new columns (like goal from bmi)

Join related tables (intake + food)

Save clean outputs to /parquet/silver/

Data masking for age
'''

'Task is to\nClean columns (trim, fix cases, remove duplicates)\n\nConvert datatypes\n\nDerive new columns (like goal from bmi)\n\nJoin related tables (intake + food)\n\nSave clean outputs to /parquet/silver/\n\nData masking for age\n'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, split, concat_ws, col, current_timestamp

In [3]:
spark=SparkSession.builder.appName("Silverlayertransform").getOrCreate()

In [ ]:
spark.conf.set("spark.sql.session.timeZone", "Asia/Kolkata")

In [ ]:
BASE = "/home/jovyan/work"
BRONZE = f"{BASE}/parquet/bronze"
SILVER = f"{BASE}/parquet/silver"

In [ ]:
users_b  = spark.read.parquet(f"{BRONZE}/users")
foods_b  = spark.read.parquet(f"{BRONZE}/food_items")
intake_b = spark.read.parquet(f"{BRONZE}/intake_logs")
water_b  = spark.read.parquet(f"{BRONZE}/water_logs")
try: mood_b = spark.read.parquet(f"{BRONZE}/mood_logs")
except: mood_b = None

In [ ]:
def age_band_expr(c):
    return (when(col(c) < 20, lit("<20"))
            .when((col(c) >= 20) & (col(c) <= 22), lit("20-22"))
            .when((col(c) >= 23) & (col(c) <= 26), lit("23-26"))
            .when((col(c) >= 27) & (col(c) <= 30), lit("27-30"))
            .otherwise(lit(">30")))
#Data masking no longer raw age but used age band here

In [18]:
#Trimming,converting upper to lower cases
from pyspark.sql.functions import lower
def age_band_expr(c):
    return (when(col(c) < 20, lit("<20"))
            .when((col(c) >= 20) & (col(c) <= 22), lit("20-22"))
            .when((col(c) >= 23) & (col(c) <= 26), lit("23-26"))
            .when((col(c) >= 27) & (col(c) <= 30), lit("27-30"))
            .otherwise(lit(">30")))
users_s = (
    users_b
      .withColumn("name", trim(col("name")))
      .withColumn("gender", lower(trim(col("gender"))))
      .withColumn("goals", lower(trim(col("goals"))))
      .withColumn("age", col("age").cast("int"))
      .withColumn("bmi", col("bmi").cast("double"))
      .withColumn("dob", col("dob").cast("string"))
      .withColumn(
          "goal",
          when(col("goals").isNotNull() & (col("goals") != ""), col("goals"))
          .when(col("bmi") < 18,  lit("weight_gain"))
          .when(col("bmi") >= 25, lit("weight_loss"))
          .otherwise(lit("maintenance"))
      )
      .withColumn("age_band", age_band_expr("age"))
      .withColumn("name", trim(col("name")))
      .withColumn("gender", lower(trim(col("gender"))))
      .withColumn("goals", lower(trim(col("goals"))))
      .withColumn("_processed_ts", current_timestamp())
      .drop("goals")     # old column replaced by 'goal'
      .drop("age") 
      .dropDuplicates(["user_id"])

      # 👇 New correction logic
      .withColumn(
          "gender",
          when(col("name").rlike("(?i)rahul|aditya|rohan|amit|sumedh|pratik|harsh|saurabh|kaustubh|nikhil|gopal|krishna|govind|madhav|damodar|vitthal|narayan|murlidhar"),
               lit("m"))
          .when(col("name").rlike("(?i)radharani|kishori|radhe|rukmini|satyabhama|lakshmi|shraddha|rutuja|pooja|mitali|sakshi|aishwarya|neha|ankita|priya|kavya"),
               lit("f"))
          .otherwise(col("gender"))
      )
)

In [19]:
users_s.show(truncate=False)

+-------+----------+------+----------+----+-------------------------+-------+-----------+--------+--------------------------+
|user_id|name      |gender|dob       |bmi |_ingest_ts               |_source|goal       |age_band|_processed_ts             |
+-------+----------+------+----------+----+-------------------------+-------+-----------+--------+--------------------------+
|1      |Krishna   |m     |2003-08-17|24.6|2025-10-30 14:50:48.78916|users  |maintenance|20-22   |2025-10-30 18:59:55.501925|
|2      |Radharani |f     |2002-03-31|20.0|2025-10-30 14:50:48.78916|users  |maintenance|23-26   |2025-10-30 18:59:55.501925|
|3      |Kishori   |f     |2000-10-01|19.6|2025-10-30 14:50:48.78916|users  |maintenance|23-26   |2025-10-30 18:59:55.501925|
|4      |Jagannath |m     |1999-11-15|21.3|2025-10-30 14:50:48.78916|users  |maintenance|23-26   |2025-10-30 18:59:55.501925|
|5      |Vitthal   |m     |2002-09-06|18.8|2025-10-30 14:50:48.78916|users  |maintenance|23-26   |2025-10-30 18:59:55.

In [4]:
from pyspark.sql.functions import col, trim, lower, to_timestamp, round as sround, lit, when
import os

BASE   = "/home/jovyan/work"
BRONZE = f"{BASE}/parquet/bronze"
SILVER = f"{BASE}/parquet/silver"

# 1️⃣ FOOD ITEMS  → silver/food_items
foods_b = spark.read.parquet(f"{BRONZE}/food_items")
foods_s = (
    foods_b
      .dropDuplicates(["food_id"])
      .select(
          col("food_id").cast("int"),
          trim(lower(col("name"))).alias("food_name"),
          trim(col("serving_unit")).alias("serving_unit"),
          col("protein_g").cast("double"),
          col("carbs_g").cast("double"),
          col("fat_g").cast("double"),
          col("kcal_per_serving").cast("double")
      )
)
foods_s.write.mode("overwrite").parquet(f"{SILVER}/food_items")

# 2️⃣ WATER LOGS → silver/water_logs
water_b = spark.read.parquet(f"{BRONZE}/water_logs")
water_s = (
    water_b
      .withColumn("user_id", col("user_id").cast("int"))
      .withColumn("ml", col("ml").cast("int"))
      .withColumn("taken_at", to_timestamp("taken_at"))
)
water_s.write.mode("overwrite").parquet(f"{SILVER}/water_logs")

# 3️⃣ INTAKE LOGS → silver/intake_enriched
intake_b = spark.read.parquet(f"{BRONZE}/intake_logs")
intake_s = (
    intake_b
      .withColumn("user_id", col("user_id").cast("int"))
      .withColumn("food_id", col("food_id").cast("int"))
      .withColumn("servings", col("servings").cast("double"))
      .withColumn("taken_at", to_timestamp("taken_at"))
      .join(foods_s, "food_id", "left")
      .withColumn("kcal",      sround(col("servings") * col("kcal_per_serving"), 2))
      .withColumn("protein_g", sround(col("servings") * col("protein_g"), 2))
      .withColumn("carbs_g",   sround(col("servings") * col("carbs_g"), 2))
      .withColumn("fat_g",     sround(col("servings") * col("fat_g"), 2))
)
intake_s.write.mode("overwrite").parquet(f"{SILVER}/intake_enriched")

print("✅ Silver rebuild complete.")
print("silver:", os.listdir(SILVER))


✅ Silver rebuild complete.
silver: ['food_items', 'intake_enriched', 'users', 'water_logs']


In [5]:
print(os.listdir(SILVER))

['food_items', 'intake_enriched', 'users', 'water_logs']


In [ ]:
'''#Casting
covert_datatypes=(
    users_b
    .withColumn("age", col("age").cast("int"))
    .withColumn("bmi", col("bmi").cast("double"))
    .withColumn("dob", col("dob").cast("string"))
)'''


In [21]:
users_s.write.mode("overwrite").parquet(f"{SILVER}/users")
# foods_s.write.mode("overwrite").parquet(f"{SILVER}/food_items")
# intake_s.write.mode("overwrite").parquet(f"{SILVER}/intake_enriched")
# water_s.write.mode("overwrite").parquet(f"{SILVER}/water_logs")

print("✅ Users Silver dataset written successfully!")

✅ Users Silver dataset written successfully!


In [10]:
'''derive goal using BMI:

bmi < 18 → weight gain

bmi ≥ 25 → weight loss

otherwise → maintenance
'''
from pyspark.sql.functions import when, lit, col
goal = (
    users_b
      .withColumn(
          "goal",
          when(col("goals").isNotNull() & (col("goals") != ""), col("goals"))
          .when(col("bmi") < 18,  lit("weight_gain"))
          .when(col("bmi") >= 25, lit("weight_loss"))
          .otherwise(lit("maintenance"))
      )
      .withColumn("age_band", age_band_expr("age"))
)


In [11]:
#users_s = users_s.dropDuplicates(["user_id"])

In [12]:
#Some cases name were male names but in geneder f was written just to correct that we are using again literal value
'''from pyspark.sql.functions import when

# Define gender correction rules
users_s = (
    users_b
      .withColumn("name", trim(col("name")))
      .withColumn("gender", lower(trim(col("gender"))))
      .withColumn("goals", lower(trim(col("goals"))))

      # 👇 New correction logic
      .withColumn(
          "gender",
          when(col("name").rlike("(?i)rahul|aditya|rohan|amit|sumedh|pratik|harsh|saurabh|kaustubh|nikhil|gopal|krishna|govind|madhav|damodar|vitthal|narayan|murlidhar"),
               lit("m"))
          .when(col("name").rlike("(?i)radharani|kishori|radhe|rukmini|satyabhama|lakshmi|shraddha|rutuja|pooja|mitali|sakshi|aishwarya|neha|ankita|priya|kavya"),
               lit("f"))
          .otherwise(col("gender"))
      )
)'''


'from pyspark.sql.functions import when\n\n# Define gender correction rules\nusers_s = (\n    users_b\n      .withColumn("name", trim(col("name")))\n      .withColumn("gender", lower(trim(col("gender"))))\n      .withColumn("goals", lower(trim(col("goals"))))\n\n      # 👇 New correction logic\n      .withColumn(\n          "gender",\n          when(col("name").rlike("(?i)rahul|aditya|rohan|amit|sumedh|pratik|harsh|saurabh|kaustubh|nikhil|gopal|krishna|govind|madhav|damodar|vitthal|narayan|murlidhar"),\n               lit("m"))\n          .when(col("name").rlike("(?i)radharani|kishori|radhe|rukmini|satyabhama|lakshmi|shraddha|rutuja|pooja|mitali|sakshi|aishwarya|neha|ankita|priya|kavya"),\n               lit("f"))\n          .otherwise(col("gender"))\n      )\n)'

In [14]:
users_s.show(truncate=False)

NameError: name 'users_s' is not defined